# Tutorial 3 - Calculating mineral-fluid equilibrium with pyDEW

First import the required packages:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyDEW

## Calculate fluid composition and speciation for a fluid in equilibrium with a rock

More realistic calculations involve equilibrium with minerals. Every element in the system that is not directly specified by molality must be specified by equilibrium with a mineral or solid solution. fO2 and pH are set to a default value or by charge balance unless directly specified.

### Quartz solubility in brine

Create an instance of the default system

In [2]:
dew_system = pyDEW.System()

Set up the elements of the fluid not constrained by mineral equilibria:

In [3]:
molalities = {'NA+':0.05,
              'CL-':0.05,
              }
T = 300.0 + 273.15 # in K
P = 10000.0 # in bar

Tell EQ3 to set the Si content of the solution by equilibrium with Quartz:

In [4]:
mineral_eq = {'H4SIO4(AQ)':'QUARTZ_ALPHA'}

Create the fluid:

In [5]:
dew_fluid = pyDEW.Fluid(dew_system,T,P,molalities=molalities,mineral_eq=mineral_eq)

First we can check that quartz is the stable mineral phase at these conditions:

In [6]:
dew_fluid.mineral_saturation

,mineral,log_q_k,aff,state
0,QUARTZ_ALPHA,0.000,0.000,satd
1,QUARTZ_BETA,-0.120,-0.316,satd
2,COESITE,-0.189,-0.496,satd
3,HALITE,-4.836,-12.684,NaN
4,CRISTO_ALPHA,-0.435,-1.140,NaN
5,CRISTO_BETA,-0.558,-1.463,NaN


Neither coesite, nor either of the cristobalite polymorphs are stable. The calculation reports that beta-quartz is stable, but this is because the affinity is very close to zero.

In [7]:
dew_fluid.elemental_comp

,ppm,molality
element,,
O,889680.9271,55.607143
H,111991.4517,111.113654
SI,706.5789,0.025158
CL,1772.6500,0.050000
NA,1149.4885,0.050000


But in what species is the Si dissolved?

In [8]:
dew_fluid.aqueous_species

,species,molality,log_g,activity
0,NA+,4.956025e-02,-0.0972,3.962000e-02
1,CL-,4.955870e-02,-0.0972,3.962000e-02
2,H4SIO4(AQ),2.067283e-02,0.0000,2.067000e-02
3,H6SI2O7(AQ),2.226384e-03,0.0000,2.226000e-03
4,NACL(AQ),4.386328e-04,0.0000,4.386000e-04
5,H+,8.535219e-05,-0.0972,6.824000e-05
6,OH-,5.436728e-05,-0.0972,4.347000e-05
7,H3SIO4-,3.252652e-05,-0.0972,2.601000e-05
8,HCL(AQ),2.663682e-06,0.0000,2.664000e-06
9,NAOH(AQ),1.122067e-06,0.0000,1.122000e-06


And what's the pH?

In [9]:
dew_fluid.pH

4.166

### A fluid in equilibrium with Fayalite-Magnetite-Quartz fO2 buffer

In this example we will allow the fO2 of the fluid to be set by equilibrium with a mineral assemblage. The system involves both Fe and Si, so both of these must also be set:

In [10]:
mineral_eq = {'H4SIO4(AQ)':'QUARTZ_ALPHA',
              'FE+2':'FAYALITE',
              'O2(G)':'MAGNETITE'}
T = 300.0 + 273.15 # in K
P = 10000.0 # in bar

This time we won't add any NaCl.

In [11]:
dew_fluid = pyDEW.Fluid(dew_system, T, P, mineral_eq = mineral_eq)

Hopefully the code has correctly identified that the fO2 is being set by mineral equilibria, and so it hasn't used the abitrary value, let's check:

In [12]:
dew_fluid.fO2

-34.133

Let's check if any other minerals are saturated:

In [13]:
dew_fluid.mineral_saturation

,mineral,log_q_k,aff,state
0,MAGNETITE,0.000,0.000,satd
1,HEMATITE,-0.708,-1.858,NaN
2,FAYALITE,0.000,0.000,satd
3,FERROSILITE,0.052,0.135,satd
4,QUARTZ_ALPHA,0.000,0.000,satd
5,QUARTZ_BETA,-0.120,-0.316,satd
6,COESITE,-0.189,-0.496,satd
7,IRON_ALPHA,-4.529,-11.878,NaN
8,IRON_GAMMA,-4.786,-12.552,NaN
9,CRISTO_ALPHA,-0.435,-1.140,NaN


Ferrosilite is saturated. Perhaps this isn't a surprise that the FMQ buffer isn't stable everywhere, but is instead a useful reference point...

### Fluids in equilibrium with solid solutions

We can see what the chemistry of a fluid in equilibrium with a fictional harzburgite. First let's see what the default solid solutions built into the module are:

In [14]:
pyDEW.defaultsystem.solid_solutions

{'KFELDSPAR(SS)': ['KFELDSPAR', 'ALBITE'],
 'PLAGIOCLASE(SS)': ['ALBITE', 'ANORTHITE'],
 'ORTHOPYROXENE(SS)': ['ENSTATITE_OR', 'FERROSILITE'],
 'OLIVINE(SS)': ['FORSTERITE', 'FAYALITE'],
 'BIOTITE(SS)': ['PHLOGOPITE', 'ANNITE'],
 'GARNET(SS)': ['PYROPE', 'ALMANDINE', 'GROSSULAR'],
 'CLINOPYROXENE(SS)': ['DIOPSIDE', 'JADEITE', 'HEDENBERGITE'],
 'CPX_SUBCALCIC(SS)': ['DIOPSIDE', 'HEDENBERGITE', 'ENSTATITE_CL'],
 'CA_AMPHIBOLE(SS)': ['TREMOLITE', 'FERROTREMOLITE'],
 'CHLORITE(SS)': ['CLINOCHLORE', 'CHAMOSITE'],
 'CALCITE(SS)': ['CALCITE', 'MAGNESITE', 'SIDERITE'],
 'APATITE(SS)': ['HYDROXYAPATITE', 'FLUORAPATITE', 'CHLORAPATITE']}

Similarly to the pure mineral equilibria, we set the species by equilibrium in the mineral_eq variable, but this time we use a list containing the name of the solid solution, and the name of the endmember.

In [15]:
mineral_eq = {'H4SIO4(AQ)': ['ORTHOPYROXENE(SS)','ENSTATITE_OR'],
              'FE+2': ['OLIVINE(SS)','FAYALITE'],
              'MG+2': ['OLIVINE(SS)','FORSTERITE'],
              'O2(G)':'MAGNETITE'}

We must also specify the solid solution composition of the rock, using a dictionary of dictionaries:

In [16]:
solid_solutions = {'OLIVINE(SS)':{'FORSTERITE':0.9,
                                  'FAYALITE': 0.1},
                   'ORTHOPYROXENE(SS)':{'ENSTATITE_OR':0.8,
                                        'FERROSILITE':0.2}}

Let's generate the fluid.

In [17]:
T = 300.0 + 273.15 # in K
P = 10000.0 # in bar
dew_fluid = pyDEW.Fluid(dew_system,T,P,mineral_eq=mineral_eq, solid_solutions=solid_solutions)

/Users/simonmatthews/opt/anaconda3/lib/python3.7/site-packages/pyDEW/output.py:150: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ss_info = pd.Series()


How much Mg, Fe, and Si are dissolved in the fluid?

In [18]:
dew_fluid.elemental_comp

,ppm,molality
element,,
O,888302.7254,55.521002
H,111906.9015,111.029766
SI,78.6151,0.002799
FE,0.1236,0.000002
MG,15.0375,0.000619


And what species are present in the fluid?

In [19]:
dew_fluid.aqueous_species

,species,molality,log_g,activity
0,H4SIO4(AQ),2.642028e-03,0.0000,2.642000e-03
1,OH-,1.022870e-03,-0.0212,9.740000e-04
2,MG+2,4.762868e-04,-0.0850,3.916000e-04
3,MG(OH)+,1.362441e-04,-0.0212,1.297000e-04
4,H2(AQ),9.997555e-05,0.0000,9.998000e-05
5,H3SIO4-,7.820924e-05,-0.0212,7.448000e-05
6,H6SI2O7(AQ),3.636427e-05,0.0000,3.636000e-05
7,MG(H3SIO4)+,6.168153e-06,-0.0212,5.874000e-06
8,H+,3.197902e-06,-0.0212,3.045000e-06
9,FE(OH)+,1.181670e-06,-0.0212,1.125000e-06


The pH?

In [20]:
dew_fluid.pH

5.5164

Is this a feasible mineral assemblage?

In [21]:
dew_fluid.mineral_saturation

,mineral,log_q_k,aff,state
0,MAGNETITE,0.0,0.0,satd
1,HEMATITE,-0.655,-1.718,NaN
2,PERICLASE,-2.364,-6.201,NaN
3,BRUCITE,-0.412,-1.081,NaN
4,FORSTERITE,-0.046,-0.12,satd
5,FAYALITE,-1.0,-2.623,NaN
6,CHRYSOTILE,2.104,5.517,ssatd
7,ENSTATITE_CL,-0.057,-0.149,satd
8,ENSTATITE_OR,-0.097,-0.254,satd
9,ENSTATITE_PR,-0.27,-0.709,NaN


Well, it looks like a fluid infiltrating the rock will react with the mineral assemblage to form some hydrated phases... This seems likely to happen in reality! This could be a good starting point for an EQ6 reaction path calculation...